# Send torch model over RabbitMQ


## for MacOS

```bash

# 설치
$ brew update
$ brew install rabbitmq

# rabbitmq를 서비스로 꾸준히 실행하고자 할 때
$ brew services start rabbitmq

# rabbitmq를 한번만 실행하고자 할 때 
$ CONF_ENV_FILE="/opt/homebrew/etc/rabbitmq/rabbitmq-env.conf" /opt/homebrew/opt/rabbitmq/sbin/rabbitmq-server

# 브라우저 열어서 어드민 UI 접속
$ open http://localhost:15672
   guest/guest

# 콘솔에서 사용자 추가
$ rabbitmqctl add_user <username> <password>
$ rabbitmqctl set_user_tags <username> administrator

# 파이썬에서 rabbitmq사용하기 위해서 pika 패키지 설치
$ !pip install pika

```


## 자주쓰는 rabbitmqctl 명령어 

```bash
# rabbitmq 서비스 시작 / 재시작 /종료
$ rabbitmqctl start / restart / stop

# rabbitmq 서비스 상태 확인
$ rabbitmqctl status

# rabbitmq 사용자 목록 표시
$ rabbitmqctl list_users

# 사용자 생성
$ rabbitmqctl add_user <유저명> <비밀번호>

# 사용자 삭제
$ rabbitmqctl delete_user <유저명>

# 사용자 태그 할당
$ rabbitmqctl set_user_tags <유저명> <태그>

# management 플러그인 - 어드민 계정 만들기
$ rabbitmqctl set_user_tags admin administrator

# Queue에 차있는 메시지 확인 방법 1
$ rabbitmqctl list_queues

# Queue에 차있는 메시지 확인 방법 2
$ rabbitmqadmin list queues


```


## 중요사항

- RabbitMQ 기본포트 번호는 5672 입니다.



## send.py

In [16]:
#!/usr/bin/env python
import pika

connection = pika.BlockingConnection(pika.ConnectionParameters(host='localhost'))
channel = connection.channel()


channel.queue_declare(queue='hello')

channel.basic_publish(exchange='',
                      routing_key='hello',
                      body='Hello World!')
print(" [x] Sent 'Hello World!'")
connection.close()

 [x] Sent 'Hello World!'


## receive.py

- 메시지를 계속 기다립니다.

In [41]:
#!/usr/bin/env python
import pika

#!/usr/bin/env python
import pika

def main():
    connection = pika.BlockingConnection(pika.ConnectionParameters(host='localhost', port=5672))
    channel = connection.channel()

    channel.queue_declare(queue='hello', durable=False)

    def callback(ch, method, properties, body):
        print(" [d] Received %r" % body)

    channel.basic_consume(queue='hello', on_message_callback=callback, auto_ack=True)
    print('[*] Waiting for messages. To exit press CTRL+C in console, or Interrupted Kernel Button in jupyter-notebook')
    channel.start_consuming()

if __name__ == '__main__':
    try:
        main()
    except KeyboardInterrupt:
        print('Interrupted')


[*] Waiting for messages. To exit press CTRL+C
Interrupted


## send.py with user info

In [33]:
import pika

userInfo = pika.PlainCredentials('guest','guest')
connection = pika.BlockingConnection(pika.ConnectionParameters(host = 'localhost', port = 5672, credentials = userInfo))
channel = connection.channel()

channel.queue_declare(queue='hello')

channel.basic_publish(exchange='',
                      routing_key='hello',
                      body='Hello World!')
print(" [x] Sent 'Hello World!'")
connection.close()

 [x] Sent 'Hello World!'


In [23]:
! rabbitmqctl add_user guest2 guest2

Adding user "guest2" ...
Done. Don't forget to grant the user permissions to some virtual hosts! See 'rabbitmqctl help set_permissions' to learn more.


In [29]:
! rabbitmqctl set_user_tags guest2 administrator

Setting tags for user "guest2" to [administrator] ...


In [37]:
! rabbitmqctl add_user guest3 guest3

Adding user "guest3" ...
Done. Don't forget to grant the user permissions to some virtual hosts! See 'rabbitmqctl help set_permissions' to learn more.


In [38]:
! rabbitmqctl set_permissions -p / "guest3" ".*" ".*" ".*"

Setting permissions for user "guest3" in vhost "/" ...


In [39]:
import pika

userInfo = pika.PlainCredentials('guest3','guest3')
connection = pika.BlockingConnection(pika.ConnectionParameters(host = 'localhost', port = 5672, credentials = userInfo))
channel = connection.channel()

channel.queue_declare(queue='hello')

channel.basic_publish(exchange='',
                      routing_key='hello',
                      body='Hello World!')
print(" [x] Sent 'Hello World!'")
connection.close()

 [x] Sent 'Hello World!'


In [43]:
! rabbitmqadmin list queues

+-------+----------+
| name  | messages |
+-------+----------+
| hello | 0        |
+-------+----------+
